<a href="https://colab.research.google.com/github/luciaPi/Neural-networks-course/blob/main/4Uloha_trening_neuronova_siet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Predikcia kvality vína**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data_utils
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

## **1. Príprava dát**
**Red Wine Quality dataset:** https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009

In [ ]:
# !ls /root/.cache/kagglehub/datasets/uciml/red-wine-quality-cortez-et-al-2009/versions/2

import kagglehub

# Download latest version
path = kagglehub.dataset_download("uciml/red-wine-quality-cortez-et-al-2009")

print("Path to dataset files:", path)
data = pd.read_csv(path+"/winequality-red.csv")
data.head()

In [ ]:
# graf kvality vina
sns.countplot(x = 'quality', data=data)

In [ ]:
# oddelime vstupy a vystup
x_data = data.iloc[:, 0: -1]
y_data = data.iloc[:, -1]

# upravime (normalizacia) vstupy
scaler = MinMaxScaler()
x_data = scaler.fit_transform(x_data)
x_data, y_data = np.array(x_data), np.array(y_data).astype(float)

# upravime data na tenzory
x_data = torch.from_numpy(x_data).float()
y_data = torch.from_numpy(y_data).float()

In [ ]:
# pripravime dataloader
batch_size = ???
train_tensor = data_utils.TensorDataset(x_data, y_data)
train_dataloader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

In [ ]:
# prvy batch dat
next(iter(train_dataloader))

## **2. Príprava neurónovej siete**

In [ ]:
epochs = ???
learning_rate = ???
num_attributes = ???

# prioritne budeme trenovat na GPU karte
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# postavime neuronovu siet
class WineModel(nn.Module):
    def __init__(self, num_attributes):
        super().__init__()
        self.layer_1 = nn.Linear(num_attributes, 16)  # torch.nn.Linear(in_features, out_features)
        self.layer_2 = nn.Linear(16, 32)
        self.layer_3 = nn.Linear(32, 16)
        self.layer_out = nn.Linear(16, 1)

    def forward(self, x):  # x = vstup
        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_out(x)
        return x

# vyrvorime neuronovu siet
model = WineModel(num_attributes).to(device).train()
print(model)

In [ ]:
criterion = ???
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## **3. Tréning neurónovej siete**

In [ ]:
# treningovy cyklus
print("Start training")

absolute_errors = []
losses_squarred_errors = []

# trening prebieha v niekolkych epochach
for epoch in range(epochs): # pocas jednej epochy prejdeme cely dataset
    # inicializujeme pre treningove statistiky
    running_loss_mse = 0.0
    running_mae = 0.0
    batches = 0

    # v cykle prejdeme vsetky data (po batchoch)
    for i, data in enumerate(train_dataloader):
        inputs, labels = data
        batches+=1

        # reshape data
        inputs = inputs.view(-1,num_attributes)
        labels = labels.view(-1,1)

        # restartujeme optimizer
        ???

        # vypocitame vystup siete pre aktualny batch dat
        ???

        # priemerna absolutna chyba (MAE=mean absolute error)
        MAE = abs(labels-y).mean()
        running_mae += MAE.item()

        # vypocitame chybu siete - trening sa ju snazi zlepsit (MSE=mean squarred error)
        ???

        # uprava parametrov siete (vahy) podla aktualnej chyby (loss)
        ???

        running_loss_mse += loss.item()

    # priemerna chyba za celu epochu
    MSE = running_loss_mse/batches
    losses_squarred_errors.append(MSE)
    MAE = running_mae/batches
    absolute_errors.append(MAE)

    print('epoch {0}: MSE={1} MAE={2}'.format(epoch+1,MSE,MAE))

print("Finished training")

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(10,4),constrained_layout=True)
axes[0].plot(losses_squarred_errors, label='Training Loss MSE')
axes[1].plot(absolute_errors, label='MAE')
axes[0].set_xlabel('Epochs')
axes[1].set_xlabel('Epochs')
axes[0].set_title('Training Loss MSE')
axes[1].set_title('MAE')
plt.show()

In [ ]:
# nastavime model do evaluacneho (netreningoveho modu)
model.eval()

# vystup celeho datasetu
y = model(x_data).detach().numpy()[:,0]

# priemerna absolutna chyba (MAE=mean absolute error)
MAE = abs(y_data-y).mean().item()

# priemerna stvorcova chyba (MSE=mean squarred error)
MSE = ((y_data-y)**2).mean().item()

print("MAE =",MAE)
print("MSE =",MSE)

In [ ]:
# overte model pre konkretnu vzorku vina
index_vstupu = ???  # vyberte vzorku vina (index)
print("Vstup =",x_data[index_vstupu])
y = model(x_data[index_vstupu]).item()
print("Vystup =",y)
print("Pozadovany vystup =",y_data[index_vstupu].item())
chyba1 = abs(y-y_data[index_vstupu].item())
print("Absolutna chyba =", chyba1)
chyba2 = (y-y_data[index_vstupu].item())**2
print("Stvorcova chyba =", chyba2)